In [1]:
import pandas as pd
import csv,os
import time
from datetime import datetime, date, time, timedelta
import numpy as np

##### d1=store master
##### d2=Item Master
##### d3=Closing Stock as on_dd.mm.yy ---> Closing Stock as on_01.12.19 

In [2]:
#Define you directory
data_dir = '/Users/nirajkulkarni/Desktop/Niraj/ISB-CBA/Capstone/data/'

d1 = pd.read_excel(data_dir + 'master/store master.xlsx')
d2 = pd.read_excel(data_dir + 'master/Item Attributes_Encoded_final.xlsx')
stock_file = data_dir + 'weekly-closing-stock/Weekly Stock as on_11.06.17.xlsx' # Closing Stock as on_01.12.19.xlsx'  #
d3 = pd.read_excel(stock_file) #Closing Stock as on_01.12.19.xlsx') #')
d4 = pd.read_excel(data_dir + 'master/market share - encoded.xlsx')

In [3]:
#read all sales file
sale_17_18 = pd.read_excel(data_dir + 'yearly-transactions/2017-18/Sale Data 01.04.17 to 31.03.18.xlsx')
sale_17_18.columns = ['Store No','Date','Item No', 'Brand','Department','Customer No', 'Quantity','Price',
                      'Total Price','Line Discount Amount', 'CRM Line Disc_ Amount', 'Discount Amount',
                      'Tax Amount' ,'Cost Amount', 'Billing', 'Contribution', 'Receipt Date','Trade Incentive' ,
                      'Trade Incentives Value', 'Total Contribution', 'State','Region'] 

In [4]:
sale_18_19 = pd.read_excel(data_dir + 'yearly-transactions/2018-19/Sale Data 01.04.18 to 31.03.19.xlsx')
sale_18_19.columns = ['Store No','Date','Item No', 'Brand','Department','Customer No', 'Quantity','Price',
                      'Total Price','Line Discount Amount', 'CRM Line Disc_ Amount', 'Discount Amount',
                      'Tax Amount' ,'Cost Amount', 'Billing', 'Contribution', 'Receipt Date','Trade Incentive' ,
                      'Trade Incentives Value', 'Total Contribution', 'State','Region'] 

In [5]:
sale_19_20 = pd.read_excel(data_dir + 'yearly-transactions/2019-20/Sale Data-Encoded 01.04.19 to 31.03.2020 - Copy.xlsx') 
sale_19_20.columns = ['Store No','Date','Item No', 'Brand','Department','Customer No', 'Quantity','Price',
                      'Total Price','Line Discount Amount', 'CRM Line Disc_ Amount', 'Discount Amount',
                      'Tax Amount' ,'Cost Amount', 'Billing', 'Contribution', 'Receipt Date','Trade Incentive' ,
                      'Trade Incentives Value', 'Total Contribution', 'State','Region'] 


In [6]:
#concat all sales file
all_sales_list = [sale_17_18,sale_18_19,sale_19_20]
all_sales = pd.concat(all_sales_list)

In [7]:
print(sale_17_18.shape)
print(sale_18_19.shape)
print(sale_19_20.shape)
print(all_sales.shape)

(75181, 22)
(76956, 22)
(72201, 22)
(224338, 22)


In [23]:
all_sales.head(2)
#all_sales[(all_sales['Item No'] == 5121685) & (all_sales['Store No'] == 'S07')]

,Store No,Date,Item No,Brand,Department,Customer No,Quantity,Price,Total Price,Line Discount Amount,...,Cost Amount,Billing,Contribution,Receipt Date,Trade Incentive,Trade Incentives Value,Total Contribution,State,Region,sales_qty_for_corp
0,S02,2017-09-30,5161847,B074,WAT,CU00247184,1,312600.0,312600.0,93800.0,...,163771.00,218800.0,7166.5,2017-06-30,0.0431,13473.06,20639.56,ST05,West,1
1,S02,2017-10-05,5174670,B074,WAT,CU00335495,1,198600.0,198600.0,39600.0,...,109391.05,159000.0,14827.7,2017-08-21,0.0431,8559.66,23387.36,ST05,West,1


In [10]:
#remove corporate sales
corporate_sales = all_sales.groupby(["Store No","Date","Item No","Customer No"])['Quantity'].agg(['sum']) #count()
corporate_sales.head(2)


sum
Store No Date       Item No Customer No     
S02      2017-04-01 5167704 CU00312748     1
                    5171321 CU00257900     1

In [16]:
corporate_sales[corporate_sales['sum'] > 2].count()


sum    883
dtype: int64

In [17]:
corporate_safes_mergedf = all_sales.merge(corporate_sales, right_index=True, how = 'inner',
                                           left_on=["Store No","Date","Item No","Customer No"],
                                           right_on=["Store No","Date","Item No","Customer No"],
                                           suffixes= ('_x', '_y')) #, indicator= True)
corporate_safes_mergedf.rename(columns = {"sum":"sales_qty_for_corp"}, inplace = True)
corporate_safes_mergedf.head(2)

,Store No,Date,Item No,Brand,Department,Customer No,Quantity,Price,Total Price,Line Discount Amount,...,Cost Amount,Billing,Contribution,Receipt Date,Trade Incentive,Trade Incentives Value,Total Contribution,State,Region,sales_qty_for_corp
0,S02,2017-09-30,5161847,B074,WAT,CU00247184,1,312600.0,312600.0,93800.0,...,163771.00,218800.0,7166.5,2017-06-30,0.0431,13473.06,20639.56,ST05,West,1
1,S02,2017-10-05,5174670,B074,WAT,CU00335495,1,198600.0,198600.0,39600.0,...,109391.05,159000.0,14827.7,2017-08-21,0.0431,8559.66,23387.36,ST05,West,1


In [18]:
all_sales = corporate_safes_mergedf

In [19]:
#create a final datafram table
final_summ_file = pd.DataFrame()

In [20]:
base_file_name = os.path.basename(stock_file)
#print(file_week)
if base_file_name.startswith("Closing"):
    file_week = base_file_name[20:26] + '20' + base_file_name[26:28]
elif base_file_name.startswith("Weekly"):
    file_week = base_file_name[19:25] + '20' + base_file_name[25:27]

file_week = datetime.strptime(file_week, '%d.%m.%Y')

end_date = file_week + timedelta(days=7)
print(file_week)
print(end_date)



#pd.to_datetime(file_week, format='%d.%m.%Y')
temp_df_date_range = all_sales[(all_sales['Date'] > file_week) & (all_sales['Date'] <= end_date)]



2017-06-11 00:00:00
2017-06-18 00:00:00


In [24]:
#temp_df_date_range.head(1)
temp_df_date_range[(temp_df_date_range['Item No'] == 5121685) & (temp_df_date_range['Store No'] == 'S07')]

,Store No,Date,Item No,Brand,Department,Customer No,Quantity,Price,Total Price,Line Discount Amount,...,Cost Amount,Billing,Contribution,Receipt Date,Trade Incentive,Trade Incentives Value,Total Contribution,State,Region,sales_qty_for_corp
10800,S07,2017-06-13,5121685,B100,WAT,CE00025313,1,31000.0,31000.0,3100.0,...,20257.61,27900.0,4109.20,2016-12-30,0.0873,2706.3,6815.50,ST09,South,4
10801,S07,2017-06-13,5121685,B100,WAT,CE00025313,1,31000.0,31000.0,3100.0,...,20257.61,27900.0,4109.20,2016-12-30,0.0873,2706.3,6815.50,ST09,South,4
10802,S07,2017-06-13,5121685,B100,WAT,CE00025313,1,31000.0,31000.0,3100.0,...,20257.60,27900.0,4109.21,2016-12-30,0.0873,2706.3,6815.51,ST09,South,4
10803,S07,2017-06-13,5121685,B100,WAT,CE00025313,1,31000.0,31000.0,3100.0,...,20257.60,27900.0,4109.21,2016-12-30,0.0873,2706.3,6815.51,ST09,South,4


In [22]:
d3.head(1)

,Location Code,Item No_,Brand,Department,Quantity,Cost Amount,Purchase MRP,Prevailing MRP as on Stock Date,Purchase Date,State,Region
0,S02,5106788,B063,WAT,1,40571.79,67500.0,72000.0,2014-02-02,ST05,West


In [25]:
#distinct store code/location code and item no for both in stock and sales data so that we do no miss any item-locs. 
#Earlier we took only values from stock, which made item-loc when we merged sales data. So idenfying all the items initially for that week
d3_unique = d3[["Location Code","Item No_"]].drop_duplicates(["Location Code","Item No_"])
d3_unique.rename(columns = {"Location Code":"Store No"}, inplace = True)
d3_unique.rename(columns = {"Item No_":"Item No"}, inplace = True)
sales_unique = temp_df_date_range[["Store No","Item No"]].drop_duplicates(["Store No","Item No"])


In [29]:
print(d3_unique[(d3_unique['Item No'] == 5121685) & (d3_unique['Store No'] == 'S07')])
print(sales_unique[(sales_unique['Item No'] == 5121685) & (sales_unique['Store No'] == 'S07')])

     Store No  Item No
4782      S07  5121685
      Store No  Item No
10800      S07  5121685


In [31]:
#merge file d3_unique and sales_unique to find all the item nos transacted in that week
all_item_in_week = pd.concat([d3_unique, sales_unique],axis=0)
print(d3_unique.shape)
print(sales_unique.shape)
print(all_item_in_week.shape)
all_item_in_week.head(2)
all_item_in_week[(all_item_in_week['Item No'] == 5121685) & (all_item_in_week['Store No'] == 'S07')]

(29585, 2)
(1280, 2)
(30865, 2)


,Store No,Item No
4782,S07,5121685
10800,S07,5121685


In [33]:
all_item_in_week = all_item_in_week.drop_duplicates(['Store No','Item No'])
print(all_item_in_week.shape)
all_item_in_week.head(1)
all_item_in_week[(all_item_in_week['Item No'] == 5121685) & (all_item_in_week['Store No'] == 'S07')]

(29787, 2)


,Store No,Item No
4782,S07,5121685


In [34]:
d3.head(2)

,Location Code,Item No_,Brand,Department,Quantity,Cost Amount,Purchase MRP,Prevailing MRP as on Stock Date,Purchase Date,State,Region
0,S02,5106788,B063,WAT,1,40571.79,67500.0,72000.0,2014-02-02,ST05,West
1,S02,5106817,B063,WAT,1,45133.00,63140.0,72000.0,2013-12-26,ST05,West


In [35]:
#New Col- Day_in_Stock
#d3_new = d3
d3['days_in_stock_init'] =  file_week - d3['Purchase Date']

In [37]:
#instead of agg, have to to use this logic to avoid error of aggreagte timedelta type
days_in_stock_aggdf = d3.groupby(["Item No_","Location Code"])['days_in_stock_init'].agg(['sum', 'count']) 
days_in_stock_aggdf['days_in_stock_init'] = days_in_stock_aggdf['sum']/days_in_stock_aggdf['count']
days_in_stock_aggdf.head()


sum  count days_in_stock_init
Item No_ Location Code                                    
5100306  S60           1225 days      1          1225 days
5100311  S06            315 days      1           315 days
         S28            315 days      1           315 days
         S62            255 days      1           255 days
         S66            162 days      1           162 days

In [38]:
merged_days_in_stock_df = d3.merge(days_in_stock_aggdf, right_index=True, how = 'inner',
                                                 left_on=["Item No_","Location Code"], right_on=["Item No_","Location Code"],
                                        suffixes= ('_x', '_y')) #, indicator= True)
merged_days_in_stock_df[(merged_days_in_stock_df['Item No_'] == 5175335) & (merged_days_in_stock_df['Location Code'] == 'S02')]


,Location Code,Item No_,Brand,Department,Quantity,Cost Amount,Purchase MRP,Prevailing MRP as on Stock Date,Purchase Date,State,Region,days_in_stock_init_x,sum,count,days_in_stock_init_y
1117,S02,5175335,B096,WAT,1,80088.59,153300.0,153300.0,2017-04-06,ST05,West,66 days,66 days,1,66 days


In [39]:
#rename column
merged_days_in_stock_df.rename(columns = {'days_in_stock_init_y':'days_in_stock_agg'}, inplace = True)
d3 = merged_days_in_stock_df


In [42]:
d3[(d3['Item No_'] == 5121685) & (d3['Location Code'] == 'S07')]

,Location Code,Item No_,Brand,Department,Quantity,Cost Amount,Purchase MRP,Prevailing MRP as on Stock Date,Purchase Date,State,Region,days_in_stock_init_x,sum,count,days_in_stock_agg
4782,S07,5121685,B100,WAT,1,81030.42,31000.0,31000.0,2016-10-17,ST09,South,237 days,812 days,4,203 days
4783,S07,5121685,B100,WAT,1,81030.42,31000.0,31000.0,2016-11-17,ST09,South,206 days,812 days,4,203 days
4784,S07,5121685,B100,WAT,1,81030.42,31000.0,31000.0,2016-11-17,ST09,South,206 days,812 days,4,203 days
4785,S07,5121685,B100,WAT,1,81030.42,31000.0,31000.0,2016-12-30,ST09,South,163 days,812 days,4,203 days


In [43]:
#d3.groupby(["Location Code","Item No_"]).agg(['count'])
#stock_quantity = d3[d3['Quantity'] > 0].groupby(["Location Code","Item No_"])['Quantity'].count()
stock_quantitydf = d3.groupby(["Location Code","Item No_"])['Quantity'].agg(['sum']) #count()
#stock_quantitydf = stock_quantitydf.to_frame()
#print(type(stock_quantity))
stock_quantitydf.head(1)

,,sum
Location Code,Item No_,
S02,5106788,1


In [44]:
merge_quantity = all_item_in_week.merge(stock_quantitydf, right_index=True, how = 'left',
                                        left_on=['Store No','Item No'], right_on=['Location Code','Item No_'], 
                                        suffixes= ('_m', '_n'))
#merge_quantity.head(4)
merge_quantity.rename(columns={'sum': 'stock_quantity'}, inplace=True)
merge_quantity[merge_quantity['Item No'] == 5175335] 

,Store No,Item No,stock_quantity
1117,S02,5175335,1.0
4262,S06,5175335,2.0
5377,S07,5175335,1.0
6750,S08,5175335,1.0
10368,S13,5175335,1.0
11556,S15,5175335,1.0
13241,S23,5175335,1.0
14686,S24,5175335,1.0
16995,S28,5175335,1.0
18152,S31,5175335,1.0


In [46]:
merge_quantity[(merge_quantity['Item No'] == 5121685) & (merge_quantity['Store No'] == 'S07')]

,Store No,Item No,stock_quantity
4782,S07,5121685,4.0


In [47]:
#Col 19 - Stock quantity needs to be grouped and counted
#TBD
#Col - 1
final_summ_file['Col_PK'] = merge_quantity['Item No'].map(str) +'_'+ merge_quantity['Store No']
final_summ_file[['Item No', 'Location Code','stock_quantity']] = merge_quantity.loc[:,['Item No', 'Store No','stock_quantity']]
#"Purchase MRP", "Prevailing MRP as on Stock Date",'State','Region',
#                 'stock_quantity','days_in_stock_agg']] = .loc[:,['Item No_', 'Location Code', 
                                                                               # "Purchase MRP", "Prevailing MRP as on Stock Date",
                                                                               # 'State','Region','days_in_stock_agg']]

final_summ_file[final_summ_file['Item No'] == 5175335] 


,Col_PK,Item No,Location Code,stock_quantity
1117,5175335_S02,5175335,S02,1.0
4262,5175335_S06,5175335,S06,2.0
5377,5175335_S07,5175335,S07,1.0
6750,5175335_S08,5175335,S08,1.0
10368,5175335_S13,5175335,S13,1.0
11556,5175335_S15,5175335,S15,1.0
13241,5175335_S23,5175335,S23,1.0
14686,5175335_S24,5175335,S24,1.0
16995,5175335_S28,5175335,S28,1.0
18152,5175335_S31,5175335,S31,1.0


In [49]:
final_summ_file[(final_summ_file['Item No'] == 5121685) & (final_summ_file['Location Code'] == 'S07')]

,Col_PK,Item No,Location Code,stock_quantity
4782,5121685_S07,5121685,S07,4.0


In [50]:
d3[d3['Item No_'] == 5175335] 

,Location Code,Item No_,Brand,Department,Quantity,Cost Amount,Purchase MRP,Prevailing MRP as on Stock Date,Purchase Date,State,Region,days_in_stock_init_x,sum,count,days_in_stock_agg
1117,S02,5175335,B096,WAT,1,80088.59,153300.0,153300.0,2017-04-06,ST05,West,66 days,66 days,1,66 days 00:00:00
4262,S06,5175335,B096,WAT,1,162694.62,153300.0,153300.0,2017-02-04,ST09,South,127 days,221 days,2,110 days 12:00:00
4263,S06,5175335,B096,WAT,1,162694.62,153300.0,153300.0,2017-03-09,ST09,South,94 days,221 days,2,110 days 12:00:00
5377,S07,5175335,B096,WAT,1,78739.50,153300.0,153300.0,2017-05-19,ST09,South,23 days,23 days,1,23 days 00:00:00
6750,S08,5175335,B096,WAT,1,80307.25,153300.0,153300.0,2017-03-09,ST02,North,94 days,94 days,1,94 days 00:00:00
10368,S13,5175335,B096,WAT,1,78739.50,153300.0,153300.0,2017-05-19,ST12,West,23 days,23 days,1,23 days 00:00:00
11556,S15,5175335,B096,WAT,1,80029.31,153300.0,153300.0,2016-12-28,ST17,South,165 days,165 days,1,165 days 00:00:00
13241,S23,5175335,B096,WAT,1,82387.37,153300.0,153300.0,2017-02-04,ST12,West,127 days,127 days,1,127 days 00:00:00
14686,S24,5175335,B096,WAT,1,81919.66,153300.0,153300.0,2016-11-03,ST16,South,220 days,220 days,1,220 days 00:00:00
16995,S28,5175335,B096,WAT,1,78739.50,153300.0,153300.0,2017-05-19,ST12,West,23 days,23 days,1,23 days 00:00:00


In [51]:
merge_d3_details = all_item_in_week.merge(d3[['Location Code','Item No_',"Purchase MRP", "Prevailing MRP as on Stock Date",'State','Region','days_in_stock_agg']], how = 'left',
                                        left_on=['Store No','Item No'], right_on=['Location Code','Item No_'], 
                                        suffixes= ('_m', '_n'))
merge_d3_details[merge_d3_details['Item No'] == 5175335] 

,Store No,Item No,Location Code,Item No_,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg
1117,S02,5175335,S02,5175335.0,153300.0,153300.0,ST05,West,66 days 00:00:00
4262,S06,5175335,S06,5175335.0,153300.0,153300.0,ST09,South,110 days 12:00:00
4263,S06,5175335,S06,5175335.0,153300.0,153300.0,ST09,South,110 days 12:00:00
5377,S07,5175335,S07,5175335.0,153300.0,153300.0,ST09,South,23 days 00:00:00
6750,S08,5175335,S08,5175335.0,153300.0,153300.0,ST02,North,94 days 00:00:00
10368,S13,5175335,S13,5175335.0,153300.0,153300.0,ST12,West,23 days 00:00:00
11556,S15,5175335,S15,5175335.0,153300.0,153300.0,ST17,South,165 days 00:00:00
13241,S23,5175335,S23,5175335.0,153300.0,153300.0,ST12,West,127 days 00:00:00
14686,S24,5175335,S24,5175335.0,153300.0,153300.0,ST16,South,220 days 00:00:00
16995,S28,5175335,S28,5175335.0,153300.0,153300.0,ST12,West,23 days 00:00:00


In [52]:
#map remaining fields from d3 i.e stock detail
final_summ_file[["Purchase MRP", "Prevailing MRP as on Stock Date",'State','Region','days_in_stock_agg']] = merge_d3_details.loc[:,["Purchase MRP", "Prevailing MRP as on Stock Date",'State','Region','days_in_stock_agg']]


In [53]:
final_summ_file[final_summ_file['Item No'] == 5175335] 

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg
1117,5175335_S02,5175335,S02,1.0,153300.0,153300.0,ST05,West,66 days 00:00:00
4262,5175335_S06,5175335,S06,2.0,153300.0,153300.0,ST09,South,110 days 12:00:00
5377,5175335_S07,5175335,S07,1.0,153300.0,153300.0,ST09,South,23 days 00:00:00
6750,5175335_S08,5175335,S08,1.0,153300.0,153300.0,ST02,North,94 days 00:00:00
10368,5175335_S13,5175335,S13,1.0,153300.0,153300.0,ST12,West,23 days 00:00:00
11556,5175335_S15,5175335,S15,1.0,153300.0,153300.0,ST17,South,165 days 00:00:00
13241,5175335_S23,5175335,S23,1.0,153300.0,153300.0,ST12,West,127 days 00:00:00
14686,5175335_S24,5175335,S24,1.0,153300.0,153300.0,ST16,South,220 days 00:00:00
16995,5175335_S28,5175335,S28,1.0,153300.0,153300.0,ST12,West,23 days 00:00:00
18152,5175335_S31,5175335,S31,1.0,153300.0,153300.0,ST15,North,165 days 00:00:00


In [54]:
final_summ_file[(final_summ_file['Item No'] == 5121685) & (final_summ_file['Location Code'] == 'S07')]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg
4782,5121685_S07,5121685,S07,4.0,31000.0,31000.0,ST09,South,203 days


In [55]:
#Col-2 Week
final_summ_file['Week_Start'] = file_week

#Col -3 Month

final_summ_file['Month'] =  file_week.month

#Col- 4 Year
final_summ_file['Year'] =  file_week.year

In [56]:
final_summ_file.dtypes

Col_PK                                      object
Item No                                      int64
Location Code                               object
stock_quantity                             float64
Purchase MRP                               float64
Prevailing MRP as on Stock Date            float64
State                                       object
Region                                      object
days_in_stock_agg                  timedelta64[ns]
Week_Start                          datetime64[ns]
Month                                        int64
Year                                         int64
dtype: object

In [57]:
final_summ_file[final_summ_file['Item No'] == 5184261]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,Month,Year


In [58]:
d1.head()

,Store Code,Store type,Store location,City type,State,Region
0,S01,Ethos,Mall,Tier2,NaN,NaN
1,S02,Ethos,Mall,Tier2,NaN,NaN
2,S03,Ethos,Mall,Tier1,NaN,NaN
3,S04,Ethos,DFS,Tier1,NaN,NaN
4,S05,Ethos,DFS,Tier1,NaN,NaN


In [59]:
merged_store_df = final_summ_file.merge(d1[['Store Code','Store type','Store location','City type']],
                                        how = 'inner', left_on=["Location Code"], right_on=["Store Code"],
                                        suffixes= ('_m', '_n')) #, indicator= True)



merged_store_df.head()

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,Month,Year,Store Code,Store type,Store location,City type
0,5106788_S02,5106788,S02,1.0,67500.0,72000.0,ST05,West,1225 days,2017-06-11,6,2017,S02,Ethos,Mall,Tier2
1,5106817_S02,5106817,S02,1.0,63140.0,72000.0,ST05,West,1263 days,2017-06-11,6,2017,S02,Ethos,Mall,Tier2
2,5106973_S02,5106973,S02,1.0,67500.0,72000.0,ST05,West,760 days,2017-06-11,6,2017,S02,Ethos,Mall,Tier2
3,5107230_S02,5107230,S02,1.0,205500.0,205500.0,ST05,West,296 days,2017-06-11,6,2017,S02,Ethos,Mall,Tier2
4,5107276_S02,5107276,S02,1.0,86000.0,86000.0,ST05,West,285 days,2017-06-11,6,2017,S02,Ethos,Mall,Tier2


In [60]:
final_summ_file  = merged_store_df
final_summ_file[final_summ_file['Item No'] == 5184261].count()

Col_PK                             0
Item No                            0
Location Code                      0
stock_quantity                     0
Purchase MRP                       0
Prevailing MRP as on Stock Date    0
State                              0
Region                             0
days_in_stock_agg                  0
Week_Start                         0
Month                              0
Year                               0
Store Code                         0
Store type                         0
Store location                     0
City type                          0
dtype: int64

In [61]:
final_summ_file[final_summ_file['Item No'] == 5184261]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,Month,Year,Store Code,Store type,Store location,City type


In [62]:
d2.head(3)

,S.No,Item No.,Brand,Department,Case Size,Case_Size,Gender,Movement,Material,Dial Color,Strap Type,Starp Color,Precioius Stone,Glass,Case Shape,Watch Type
0,1,5135192,B096,WAT,32 mm,32-34 mm,Women,Quartz,Steel & Ceramic,White,Steel & Ceramic,NaN,On Dial,Sapphire Crystal,Round,Analog
1,2,5135252,B040,WAT,34 mm,32-34 mm,Women,Automatic,Steel,MOP,Satin,Grey,On Dial,Sapphire Crystal,Round,Analog
2,3,5135301,B113,WAT,42 mm,41-43 mm,Women,Automatic,Ceramic,Pink,Leather,Pink,NaN,Sapphire Crystal,Round,Analog


In [63]:
final_summ_file[(final_summ_file['Item No'] == 5121685) & (final_summ_file['Location Code'] == 'S07')]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,Month,Year,Store Code,Store type,Store location,City type
4321,5121685_S07,5121685,S07,4.0,31000.0,31000.0,ST09,South,203 days,2017-06-11,6,2017,S07,Ethos,DFS,Tier1


In [64]:
merged_item_df = final_summ_file.merge(d2[['Item No.','Brand','Movement','Material','Strap Type',
                                           'Case Shape','Case Size','Case_Size','Gender','Dial Color','Starp Color',
                                           'Precioius Stone','Glass','Watch Type']],
                                       how = 'inner', left_on=["Item No"], right_on=["Item No."],
                                       suffixes= ('_m', '_n')) #, indicator= True)



#merged_item_df.head()

In [65]:
final_summ_file  = merged_item_df
merged_item_df[merged_item_df['Item No'] == 5106817]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Strap Type,Case Shape,Case Size,Case_Size,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type
4,5106817_S02,5106817,S02,1.0,63140.0,72000.0,ST05,West,1263 days,2017-06-11,...,Steel,Rectangular,32 mm,32-34 mm,Men,White,Silver,NaN,Sapphire Crystal,Analog
5,5106817_S06,5106817,S06,1.0,48100.0,72000.0,ST09,South,1225 days,2017-06-11,...,Steel,Rectangular,32 mm,32-34 mm,Men,White,Silver,NaN,Sapphire Crystal,Analog
6,5106817_S15,5106817,S15,1.0,62500.0,72000.0,ST17,South,1455 days,2017-06-11,...,Steel,Rectangular,32 mm,32-34 mm,Men,White,Silver,NaN,Sapphire Crystal,Analog
7,5106817_S31,5106817,S31,1.0,63140.0,72000.0,ST15,North,1225 days,2017-06-11,...,Steel,Rectangular,32 mm,32-34 mm,Men,White,Silver,NaN,Sapphire Crystal,Analog
8,5106817_S33,5106817,S33,1.0,56500.0,72000.0,ST09,South,1402 days,2017-06-11,...,Steel,Rectangular,32 mm,32-34 mm,Men,White,Silver,NaN,Sapphire Crystal,Analog
9,5106817_S62,5106817,S62,1.0,62500.0,72000.0,ST02,North,1225 days,2017-06-11,...,Steel,Rectangular,32 mm,32-34 mm,Men,White,Silver,NaN,Sapphire Crystal,Analog
10,5106817_S70,5106817,S70,1.0,67500.0,72000.0,ST12,West,913 days,2017-06-11,...,Steel,Rectangular,32 mm,32-34 mm,Men,White,Silver,NaN,Sapphire Crystal,Analog


In [66]:
final_summ_file[final_summ_file['Item No'] == 5184261].count()

Col_PK                             0
Item No                            0
Location Code                      0
stock_quantity                     0
Purchase MRP                       0
Prevailing MRP as on Stock Date    0
State                              0
Region                             0
days_in_stock_agg                  0
Week_Start                         0
Month                              0
Year                               0
Store Code                         0
Store type                         0
Store location                     0
City type                          0
Item No.                           0
Brand                              0
Movement                           0
Material                           0
Strap Type                         0
Case Shape                         0
Case Size                          0
Case_Size                          0
Gender                             0
Dial Color                         0
Starp Color                        0
P

In [ ]:
final_summ_file[final_summ_file['Item No'] == 5106817]

In [67]:
final_summ_file[(final_summ_file['Item No'] == 5121685) & (final_summ_file['Location Code'] == 'S07')]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Strap Type,Case Shape,Case Size,Case_Size,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type
319,5121685_S07,5121685,S07,4.0,31000.0,31000.0,ST09,South,203 days,2017-06-11,...,Steel,Round,41 mm,41-43 mm,Men,Black,Grey,NaN,Sapphire Crystal,Analog


In [68]:
final_summ_file.columns

Index(['Col_PK', 'Item No', 'Location Code', 'stock_quantity', 'Purchase MRP',
       'Prevailing MRP as on Stock Date', 'State', 'Region',
       'days_in_stock_agg', 'Week_Start', 'Month', 'Year', 'Store Code',
       'Store type', 'Store location', 'City type', 'Item No.', 'Brand',
       'Movement', 'Material', 'Strap Type', 'Case Shape', 'Case Size',
       'Case_Size', 'Gender', 'Dial Color', 'Starp Color', 'Precioius Stone',
       'Glass', 'Watch Type'],
      dtype='object')

In [69]:
final_summ_file.dtypes

Col_PK                                      object
Item No                                      int64
Location Code                               object
stock_quantity                             float64
Purchase MRP                               float64
Prevailing MRP as on Stock Date            float64
State                                       object
Region                                      object
days_in_stock_agg                  timedelta64[ns]
Week_Start                          datetime64[ns]
Month                                        int64
Year                                         int64
Store Code                                  object
Store type                                  object
Store location                              object
City type                                   object
Item No.                                     int64
Brand                                       object
Movement                                    object
Material                       

In [70]:
final_summ_file.head()

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Strap Type,Case Shape,Case Size,Case_Size,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type
0,5106788_S02,5106788,S02,1.0,67500.0,72000.0,ST05,West,1225 days,2017-06-11,...,Steel,Rectangular,25 mm,23-25 mm,Women,White,Silver,NaN,Sapphire Crystal,Analog
1,5106788_S08,5106788,S08,1.0,56500.0,72000.0,ST02,North,1225 days,2017-06-11,...,Steel,Rectangular,25 mm,23-25 mm,Women,White,Silver,NaN,Sapphire Crystal,Analog
2,5106788_S23,5106788,S23,1.0,72000.0,72000.0,ST12,West,528 days,2017-06-11,...,Steel,Rectangular,25 mm,23-25 mm,Women,White,Silver,NaN,Sapphire Crystal,Analog
3,5106788_S70,5106788,S70,1.0,67500.0,72000.0,ST12,West,913 days,2017-06-11,...,Steel,Rectangular,25 mm,23-25 mm,Women,White,Silver,NaN,Sapphire Crystal,Analog
4,5106817_S02,5106817,S02,1.0,63140.0,72000.0,ST05,West,1263 days,2017-06-11,...,Steel,Rectangular,32 mm,32-34 mm,Men,White,Silver,NaN,Sapphire Crystal,Analog


In [84]:
#print(temp_df_date_range.shape)
#pd.to_datetime(temp_df_date_range['Receipt Date'], format='%d.%m.%Y')
#pd.to_datetime(temp_df_date_range['Date'], format='%d.%m.%Y')
#print(temp_df_date_range.dtypes)
temp_df_date_range[temp_df_date_range['Item No'] == 5136095]
temp_df_date_range[(temp_df_date_range['Item No'] == 5121685) & (temp_df_date_range['Store No'] == 'S07')]

,Store No,Date,Item No,Brand,Department,Customer No,Quantity,Price,Total Price,Line Discount Amount,...,Cost Amount,Billing,Contribution,Receipt Date,Trade Incentive,Trade Incentives Value,Total Contribution,State,Region,sales_qty_for_corp
10800,S07,2017-06-13,5121685,B100,WAT,CE00025313,1,31000.0,31000.0,3100.0,...,20257.61,27900.0,4109.20,2016-12-30,0.0873,2706.3,6815.50,ST09,South,4
10801,S07,2017-06-13,5121685,B100,WAT,CE00025313,1,31000.0,31000.0,3100.0,...,20257.61,27900.0,4109.20,2016-12-30,0.0873,2706.3,6815.50,ST09,South,4
10802,S07,2017-06-13,5121685,B100,WAT,CE00025313,1,31000.0,31000.0,3100.0,...,20257.60,27900.0,4109.21,2016-12-30,0.0873,2706.3,6815.51,ST09,South,4
10803,S07,2017-06-13,5121685,B100,WAT,CE00025313,1,31000.0,31000.0,3100.0,...,20257.60,27900.0,4109.21,2016-12-30,0.0873,2706.3,6815.51,ST09,South,4


In [86]:
final_summ_file = final_summ_file_test

In [87]:

find_corp_salesdf = final_summ_file.merge(temp_df_date_range[['Store No','Item No','sales_qty_for_corp']],
                                                     how = 'inner',
                                                     left_on=["Location Code","Item No"],
                                                     right_on=["Store No","Item No"],
                                           suffixes= ('_x', '_y')) #, indicator= True)
#find_corp_salesdf.rename(columns = {"sum":"sales_qty_for_corp"}, inplace = True)
#find_corp_salesdf.head(4)
find_corp_salesdf['Corporate_Order'] = find_corp_salesdf['sales_qty_for_corp'].apply(lambda x: 1 if x > 2 else 0)
#final_summ_file_test = find_corp_salesdf.loc[:, find_corp_salesdf.columns != 'sales_qty_for_corp']
#remove duplicate record as we want only one record for corporate saless
find_corp_salesdf = find_corp_salesdf.drop_duplicates(["Store No","Item No"])
final_summ_file = find_corp_salesdf.drop(['sales_qty_for_corp', 'Store No'], axis=1)
final_summ_file[final_summ_file['Corporate_Order'] == 1].count()


Col_PK                             5
Item No                            5
Location Code                      5
stock_quantity                     5
Purchase MRP                       5
Prevailing MRP as on Stock Date    5
State                              5
Region                             5
days_in_stock_agg                  5
Week_Start                         5
Month                              5
Year                               5
Store Code                         5
Store type                         5
Store location                     5
City type                          5
Item No.                           5
Brand                              5
Movement                           5
Material                           5
Strap Type                         5
Case Shape                         5
Case Size                          5
Case_Size                          5
Gender                             5
Dial Color                         5
Starp Color                        5
P

In [88]:
final_summ_file[final_summ_file['Corporate_Order'] == 1]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Case Shape,Case Size,Case_Size,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order
21,5121685_S07,5121685,S07,4.0,31000.0,31000.0,ST09,South,203 days 00:00:00,2017-06-11,...,Round,41 mm,41-43 mm,Men,Black,Grey,NaN,Sapphire Crystal,Analog,1
94,5136130_S44,5136130,S44,12.0,23900.0,23900.0,ST03,North,17 days 00:00:00,2017-06-11,...,Round,43 mm,41-43 mm,Men,Black,Black,NaN,Sapphire Crystal,Analog,1
941,5180283_S83,5180283,S83,7.0,17450.0,17450.0,ST12,West,36 days 00:00:00,2017-06-11,...,Round,42 mm,41-43 mm,Men,White,Brown,NaN,Mineral Crystal,Analog,1
955,5118502_S44,5118502,S44,3.0,22800.0,22800.0,ST03,North,17 days 00:00:00,2017-06-11,...,Rectangular,25 mm,23-25 mm,Women,MOP,Silver,NaN,Sapphire Crystal,Analog,1
1325,5157411_S83,5157411,S83,8.0,16495.0,16495.0,ST12,West,296 days 18:00:00,2017-06-11,...,Round,43 mm,41-43 mm,Men,Black,Silver,NaN,Mineral Crystal,Analog,1


In [90]:
final_summ_file[(final_summ_file['Item No'] == 5121685) & (final_summ_file['Location Code'] == 'S07')]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Case Shape,Case Size,Case_Size,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order
21,5121685_S07,5121685,S07,4.0,31000.0,31000.0,ST09,South,203 days,2017-06-11,...,Round,41 mm,41-43 mm,Men,Black,Grey,NaN,Sapphire Crystal,Analog,1


In [91]:
#COl - 20 Sales QUantity
sales_quantitydf = temp_df_date_range.groupby(["Item No","Store No"])['Quantity'].agg(['sum']) #count()
#sales_quantitydf = temp_df_date_range.groupby(["Item No"])['Quantity'].agg(['sum']) #count()
print(sales_quantitydf.shape)
print(type(sales_quantitydf))
#print(sales_quantitydf.loc[:,:])


(1280, 1)
<class 'pandas.core.frame.DataFrame'>


In [92]:
sales_quantitydf.head()

sum
Item No Store No     
5107230 S07         1
        S66         1
5107236 S66         1
5107277 S02         1
        S66         1

In [93]:
#merged_sales_quantity_df = final_summ_file.merge(sales_quantitydf,how = 'inner',
#                                                 left_on=["Item No_"], right_on=["Item No"],
#                                        suffixes= ('_m', '_n')) #, indicator= True)

merged_sales_quantity_df = final_summ_file.merge(sales_quantitydf, right_index=True, how = 'left',
                                                 left_on=["Item No","Store Code"], right_on=["Item No","Store No"],
                                        suffixes= ('_m', '_n')) #, indicator= True)

merged_sales_quantity_df.rename(columns={'sum': 'sales_quantity'}, inplace=True)
merged_sales_quantity_df.head()



,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Case Size,Case_Size,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity
0,5107230_S07,5107230,S07,1.0,205500.0,205500.0,ST09,South,171 days,2017-06-11,...,38.5 mm,38-40 mm,Men,Silver,Brown,NaN,Sapphire Crystal,Analog,0,1
1,5107230_S66,5107230,S66,1.0,205500.0,205500.0,ST12,West,166 days,2017-06-11,...,38.5 mm,38-40 mm,Men,Silver,Brown,NaN,Sapphire Crystal,Analog,0,1
2,5107277_S02,5107277,S02,1.0,78000.0,78000.0,ST05,West,746 days,2017-06-11,...,24 mm,23-25 mm,Women,Black,Silver,On Dial,Sapphire Crystal,Analog,0,1
3,5107277_S66,5107277,S66,1.0,78000.0,78000.0,ST12,West,131 days,2017-06-11,...,24 mm,23-25 mm,Women,Black,Silver,On Dial,Sapphire Crystal,Analog,0,1
4,5107277_S78,5107277,S78,1.0,78000.0,78000.0,ST03,North,590 days,2017-06-11,...,24 mm,23-25 mm,Women,Black,Silver,On Dial,Sapphire Crystal,Analog,0,1


In [96]:
final_summ_file  = merged_sales_quantity_df

#print(sales_quantitydf.loc[(5184992,'S110')]) # == 5193497]]
final_summ_file[final_summ_file['Item No'] == 5193497]
# data.loc[["Avery Bradley", "R.J. Hunter"]] 
#final_summ_file[(final_summ_file['Item No'] == 5121685) & (final_summ_file['Location Code'] == 'S07')]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Case Size,Case_Size,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity
21,5121685_S07,5121685,S07,4.0,31000.0,31000.0,ST09,South,203 days,2017-06-11,...,41 mm,41-43 mm,Men,Black,Grey,NaN,Sapphire Crystal,Analog,1,4


In [97]:
#Corporate Orders
#corporate_order_df = temp_df_date_range.groupby(["Store No","Date","Item No","Customer No"])['Quantity'].agg(['sum']) #count()
#print(corporate_order_df.shape)
#print(type(corporate_order_df))
#print(corporate_order_df.head(3))
##print(corporate_order_df.loc[:,:])
#print(corporate_order_df.iloc[1:3,0:])
#print(corporate_order_df[corporate_order_df["sum"] > 2]) #.count())
#final_summ_file[final_summ_file['Col_PK'] == "5184992_S110"]

In [98]:
### TBD
#merged_corporate_order_df = final_summ_file.merge(corporate_order_df, right_index=True, how = 'left',
#                                                 left_on=["Item No_","Location Code"], right_on=["Item No","Store No"],
#                                        suffixes= ('_m', '_n')) #, indicator= True)

#merged_corporate_order_df.rename(columns={'sum': 'sales_quantity'}, inplace=True)
#merged_corporate_order_df.head()



In [99]:
#COl - Days_to_Sell
#find the different between receipt_date which is purchase date and the Data field which is sale date
temp_df_date_range['sale_diff'] = temp_df_date_range.loc[:,'Date'] -  temp_df_date_range.loc[:,'Receipt Date'] 
#instead of agg, have to to use this logic to avoid error of aggreagte timedelta type
Days_to_Sell_Aggdf = temp_df_date_range.groupby(["Item No","Store No"])['sale_diff'].agg(['sum', 'count']) 
Days_to_Sell_Aggdf['days_to_sell'] = Days_to_Sell_Aggdf['sum']/Days_to_Sell_Aggdf['count']
Days_to_Sell_Aggdf.head()

/Users/nirajkulkarni/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


sum  count days_to_sell
Item No Store No                             
5107230 S07      177 days      1     177 days
        S66      171 days      1     171 days
5107236 S66      529 days      1     529 days
5107277 S02      752 days      1     752 days
        S66      779 days      1     779 days

In [100]:
try:
    merged_days_sell_df = final_summ_file.merge(Days_to_Sell_Aggdf['days_to_sell'], right_index=True, how = 'left',
                                                left_on=["Item No","Location Code"], right_on=["Item No","Store No"],
                                                suffixes= ('_x', '_y')) #, indicator= True)
    final_summ_file  = merged_days_sell_df
except:
    print("No Data for week : " + str(file_week))
    final_summ_file['days_to_sell'] = None


In [101]:

final_summ_file[final_summ_file['Item No'] == 5136095]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Case_Size,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell
92,5136095_S08,5136095,S08,1.0,27100.0,27100.0,ST02,North,54 days,2017-06-11,...,41-43 mm,Men,White,Brown,NaN,Sapphire Crystal,Analog,0,1,57 days
93,5136095_S33,5136095,S33,1.0,27100.0,27100.0,ST09,South,188 days,2017-06-11,...,41-43 mm,Men,White,Brown,NaN,Sapphire Crystal,Analog,0,1,190 days


In [102]:
#Col - 21 AVerage BIlling Value
avg_sales_billing = temp_df_date_range.groupby(["Item No","Store No"])['Billing'].agg(['mean']) #count()
print(avg_sales_billing.shape)
print(type(avg_sales_billing))
#print(sales_quantitydf.loc[:,:])


(1280, 1)
<class 'pandas.core.frame.DataFrame'>


In [103]:
avg_sales_billing.head()

mean
Item No Store No          
5107230 S07       168500.0
        S66       164400.0
5107236 S66        58500.0
5107277 S02        60840.0
        S66        58500.0

In [104]:
merged_avg_billing_df = final_summ_file.merge(avg_sales_billing, right_index=True, how = 'left',
                                                 left_on=["Item No","Location Code"], right_on=["Item No","Store No"],
                                        suffixes= ('_m', '_n')) #, indicator= True)

merged_avg_billing_df.rename(columns={'mean': 'avg_billing'}, inplace=True)
merged_avg_billing_df.head()

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell,avg_billing
0,5107230_S07,5107230,S07,1.0,205500.0,205500.0,ST09,South,171 days,2017-06-11,...,Men,Silver,Brown,NaN,Sapphire Crystal,Analog,0,1,177 days,168500.0
1,5107230_S66,5107230,S66,1.0,205500.0,205500.0,ST12,West,166 days,2017-06-11,...,Men,Silver,Brown,NaN,Sapphire Crystal,Analog,0,1,171 days,164400.0
2,5107277_S02,5107277,S02,1.0,78000.0,78000.0,ST05,West,746 days,2017-06-11,...,Women,Black,Silver,On Dial,Sapphire Crystal,Analog,0,1,752 days,60840.0
3,5107277_S66,5107277,S66,1.0,78000.0,78000.0,ST12,West,131 days,2017-06-11,...,Women,Black,Silver,On Dial,Sapphire Crystal,Analog,0,1,779 days,58500.0
4,5107277_S78,5107277,S78,1.0,78000.0,78000.0,ST03,North,590 days,2017-06-11,...,Women,Black,Silver,On Dial,Sapphire Crystal,Analog,0,1,593 days,66300.0


In [105]:
final_summ_file  = merged_avg_billing_df
#Replace NaN with 0
final_summ_file[final_summ_file['Item No'] == 5136095]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell,avg_billing
92,5136095_S08,5136095,S08,1.0,27100.0,27100.0,ST02,North,54 days,2017-06-11,...,Men,White,Brown,NaN,Sapphire Crystal,Analog,0,1,57 days,24400.0
93,5136095_S33,5136095,S33,1.0,27100.0,27100.0,ST09,South,188 days,2017-06-11,...,Men,White,Brown,NaN,Sapphire Crystal,Analog,0,1,190 days,24932.0


In [106]:
final_summ_file[(final_summ_file['Item No'] == 5121685) & (final_summ_file['Location Code'] == 'S07')]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Gender,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell,avg_billing
21,5121685_S07,5121685,S07,4.0,31000.0,31000.0,ST09,South,203 days,2017-06-11,...,Men,Black,Grey,NaN,Sapphire Crystal,Analog,1,4,165 days,27900.0


In [107]:
#read all purchase files
purchase_17_18 = pd.read_excel(data_dir + 'yearly-transactions/2017-18/Purchase Data 01.04.17 to 31.03.18.xlsx')

In [108]:
purchase_18_19 = pd.read_excel(data_dir + 'yearly-transactions/2018-19/Purchase Data 01.04.18 to 31.03.19.xlsx')

In [109]:
purchase_19_20 = pd.read_excel(data_dir + 'yearly-transactions/2019-20/Purchase Data 01.04.19 to 06.02.2020.xlsx')

In [110]:
print(purchase_17_18.shape)
print(purchase_18_19.shape)
print(purchase_19_20.shape)

(92573, 10)
(92739, 10)
(51784, 10)


In [111]:
#concat all sales file
all_purchase_list = [purchase_17_18,purchase_18_19,purchase_19_20]
all_purchase = pd.concat(all_purchase_list)
#all_purchase['Item No_'] = all_purchase["Item No_"].astype(int)
all_purchase.dtypes

Location Code            object
Posting Date     datetime64[ns]
Item No_                float64
Brand                    object
Department               object
Quantity                float64
Purchase MRP            float64
Cost Amount             float64
State Code               object
Region                   object
dtype: object

In [112]:
all_purchase[(all_purchase['Item No_'] == 5136095) & (all_purchase['Location Code'] == 'S06')]


,Location Code,Posting Date,Item No_,Brand,Department,Quantity,Purchase MRP,Cost Amount,State Code,Region
2989,S06,2017-04-21,5136095.0,B100,WAT,1.0,27100.0,17721.42,ST09,South
28360,S06,2017-08-19,5136095.0,B100,WAT,1.0,27100.0,14926.18,ST09,South
35482,S06,2017-09-18,5136095.0,B100,WAT,1.0,27100.0,14897.56,ST09,South
35483,S06,2017-09-18,5136095.0,B100,WAT,1.0,27100.0,14897.56,ST09,South
50829,S06,2017-11-07,5136095.0,B100,WAT,1.0,27100.0,14926.17,ST09,South
50830,S06,2017-11-07,5136095.0,B100,WAT,1.0,27100.0,14926.17,ST09,South
27656,S06,2018-06-30,5136095.0,B100,WAT,1.0,26150.0,15904.58,ST09,South
34843,S06,2018-07-31,5136095.0,B100,WAT,1.0,26150.0,15623.38,ST09,South
42081,S06,2018-08-31,5136095.0,B100,WAT,1.0,26150.0,15908.88,ST09,South
42082,S06,2018-08-31,5136095.0,B100,WAT,1.0,26150.0,15908.88,ST09,South


In [113]:
purchase_temp_df_date_range = all_purchase[(all_purchase['Posting Date'] > file_week) & (all_purchase['Posting Date'] <= end_date)]
purchase_temp_df_date_range.head(3)

,Location Code,Posting Date,Item No_,Brand,Department,Quantity,Purchase MRP,Cost Amount,State Code,Region
10863,S57,2017-06-12,5162474.0,B083,WAT,-1.0,130200.0,-85106.68,ST12,West
10864,S44,2017-06-13,5180162.0,B100,WAT,1.0,45900.0,30549.38,ST03,North
10865,S65,2017-06-13,5143588.0,B083,WAT,1.0,255400.0,164876.00,ST03,North


In [114]:
purchase_quantitydf = purchase_temp_df_date_range.groupby(["Item No_","Location Code"])['Quantity'].agg(['sum'])
print(type(purchase_quantitydf))
purchase_quantitydf.head()

<class 'pandas.core.frame.DataFrame'>


,,sum
Item No_,Location Code,
5120056.0,S44,2.0
5122014.0,S43,3.0
5143588.0,S65,1.0
5158000.0,S41,1.0
5160539.0,S41,1.0


In [115]:
merged_purchase_quantity_df = final_summ_file.merge(purchase_quantitydf, right_index=True, how = 'left',
                                                 left_on=["Item No","Location Code"], right_on=["Item No_","Location Code"],
                                        suffixes= ('_i', '_j')) #, indicator= True)

merged_purchase_quantity_df.rename(columns={'sum': 'purchase_quantity'}, inplace=True)
merged_purchase_quantity_df.head()


,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell,avg_billing,purchase_quantity
0,5107230_S07,5107230,S07,1.0,205500.0,205500.0,ST09,South,171 days,2017-06-11,...,Silver,Brown,NaN,Sapphire Crystal,Analog,0,1,177 days,168500.0,NaN
1,5107230_S66,5107230,S66,1.0,205500.0,205500.0,ST12,West,166 days,2017-06-11,...,Silver,Brown,NaN,Sapphire Crystal,Analog,0,1,171 days,164400.0,NaN
2,5107277_S02,5107277,S02,1.0,78000.0,78000.0,ST05,West,746 days,2017-06-11,...,Black,Silver,On Dial,Sapphire Crystal,Analog,0,1,752 days,60840.0,NaN
3,5107277_S66,5107277,S66,1.0,78000.0,78000.0,ST12,West,131 days,2017-06-11,...,Black,Silver,On Dial,Sapphire Crystal,Analog,0,1,779 days,58500.0,NaN
4,5107277_S78,5107277,S78,1.0,78000.0,78000.0,ST03,North,590 days,2017-06-11,...,Black,Silver,On Dial,Sapphire Crystal,Analog,0,1,593 days,66300.0,NaN


In [116]:
final_summ_file = merged_purchase_quantity_df

final_summ_file[final_summ_file['Item No'] == 5136095]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell,avg_billing,purchase_quantity
92,5136095_S08,5136095,S08,1.0,27100.0,27100.0,ST02,North,54 days,2017-06-11,...,White,Brown,NaN,Sapphire Crystal,Analog,0,1,57 days,24400.0,NaN
93,5136095_S33,5136095,S33,1.0,27100.0,27100.0,ST09,South,188 days,2017-06-11,...,White,Brown,NaN,Sapphire Crystal,Analog,0,1,190 days,24932.0,NaN


In [117]:
final_summ_file[(final_summ_file['Item No'] == 5121685) & (final_summ_file['Location Code'] == 'S07')]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Dial Color,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell,avg_billing,purchase_quantity
21,5121685_S07,5121685,S07,4.0,31000.0,31000.0,ST09,South,203 days,2017-06-11,...,Black,Grey,NaN,Sapphire Crystal,Analog,1,4,165 days,27900.0,NaN


In [118]:
final_summ_file[['sales_quantity','days_to_sell','avg_billing','purchase_quantity','stock_quantity']] = final_summ_file[['sales_quantity','days_to_sell','avg_billing','purchase_quantity','stock_quantity']].fillna(0)

/Users/nirajkulkarni/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  """Entry point for launching an IPython kernel.


In [119]:
#Stock Qty + purchase Qty
final_summ_file['total_available_qty'] = final_summ_file['stock_quantity'] + final_summ_file['purchase_quantity']

In [120]:
final_summ_file[final_summ_file['Item No'] == 5136095]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell,avg_billing,purchase_quantity,total_available_qty
92,5136095_S08,5136095,S08,1.0,27100.0,27100.0,ST02,North,54 days,2017-06-11,...,Brown,NaN,Sapphire Crystal,Analog,0,1,57 days,24400.0,0.0,1.0
93,5136095_S33,5136095,S33,1.0,27100.0,27100.0,ST09,South,188 days,2017-06-11,...,Brown,NaN,Sapphire Crystal,Analog,0,1,190 days,24932.0,0.0,1.0


In [121]:
#corporate_order_df = final_summ_file.groupby(["Col_PK","Location Code"])['days_in_stock_init'].agg(['sum', 'count']) 
#purchase_quantitydf = purchase_temp_df_date_range.groupby(["Item No_","Location Code"])['Quantity'].agg(['sum'])
final_summ_file[(final_summ_file['Item No'] == 5121685) & (final_summ_file['Location Code'] == 'S07')]

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell,avg_billing,purchase_quantity,total_available_qty
21,5121685_S07,5121685,S07,4.0,31000.0,31000.0,ST09,South,203 days,2017-06-11,...,Grey,NaN,Sapphire Crystal,Analog,1,4,165 days,27900.0,0.0,4.0


In [122]:
final_summ_file.columns

Index(['Col_PK', 'Item No', 'Location Code', 'stock_quantity', 'Purchase MRP',
       'Prevailing MRP as on Stock Date', 'State', 'Region',
       'days_in_stock_agg', 'Week_Start', 'Month', 'Year', 'Store Code',
       'Store type', 'Store location', 'City type', 'Item No.', 'Brand',
       'Movement', 'Material', 'Strap Type', 'Case Shape', 'Case Size',
       'Case_Size', 'Gender', 'Dial Color', 'Starp Color', 'Precioius Stone',
       'Glass', 'Watch Type', 'Corporate_Order', 'sales_quantity',
       'days_to_sell', 'avg_billing', 'purchase_quantity',
       'total_available_qty'],
      dtype='object')

In [123]:
print((len([name for name in os.listdir("/Users/nirajkulkarni/Desktop/Niraj/ISB-CBA/Capstone/data/weekly-closing-stock") if os.path.isfile(name)])))

1


In [124]:
print(len([name for name in os.listdir("/Users/nirajkulkarni/Desktop/Niraj/ISB-CBA/Capstone/data/weekly-closing-stock")]))

162


In [125]:
summ_data = pd.read_csv('/Users/nirajkulkarni/Desktop/Niraj/ISB-CBA/Capstone/data/summarized_data.csv')
summ_data.head(2)

,Col_PK,Item No,Location Code,stock_quantity,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,days_in_stock_agg,Week_Start,...,Starp Color,Precioius Stone,Glass,Watch Type,Corporate_Order,sales_quantity,days_to_sell,avg_billing,purchase_quantity,total_available_qty
0,5107230_S07,5107230,S07,1.0,205500.0,205500.0,ST09,South,171 days 00:00:00.000000000,2017-06-11,...,Brown,NaN,Sapphire Crystal,Analog,0,1,177 days 00:00:00.000000000,168500.0,0.0,1.0
1,5107230_S66,5107230,S66,1.0,205500.0,205500.0,ST12,West,166 days 00:00:00.000000000,2017-06-11,...,Brown,NaN,Sapphire Crystal,Analog,0,1,171 days 00:00:00.000000000,164400.0,0.0,1.0


In [126]:
df = summ_data.groupby('Col_PK').filter(lambda x: len(x) > 2)
len(df)

232

In [ ]:
df